# Problemas de Satisfacción de Restricciones
- Ricardo Méndez 21289
- Sara Echeverría 21371
- Francisco Castillo 21562
- Repositorio de Github: https://github.com/bl33h/constraintSatisfactionProblems

# Task 1 - Teoría

## Algoritmo AC-3 y su relación con el algoritmo de backtracking search
El algoritmo AC-3 optimiza la resolución de problemas de satisfacción de restricciones (CSP) eliminando previamente valores que no pueden satisfacer las restricciones entre variables, reduciendo así el espacio de búsqueda. Por su parte, el algoritmo de backtracking explora este espacio simplificado para encontrar soluciones, retrocediendo cuando una ruta no conduce a una solución válida. Su relación radica en la combinación de estos, pues dan como resultado una estrategia eficaz en la cual AC-3 reduce el espacio de posibilidades y el algoritmo de backtracking navega por este espacio optimizado en busca de la solución (Gao, 2024).


## Arc Consistency
El término de ‘Arc Consistency’ describe un método eficaz para abordar problemas de satisfacción de restricciones, mediante el cual se eliminan las opciones inviables que no se ajustan a las restricciones establecidas (Brown, 2010).

## Referencias
- Brown. (2010). CSPs: Arc Consistency. The University of British Columbia. https://www.cs.ubc.ca/~kevinlb/teaching/cs322%20-%202009-10/Lectures/CSP3.pdf
- Gao. (2024). Constraint Satisfaction Problems: Backtracking Search and Arc Consistency. University of Waterloo. https://cs.uwaterloo.ca/~a23gao/cs486686_s19/slides/lec05_csp_arc_consistency_backtracking_search_nosol.pdf

# Task 2

## Restricciones
- 4 estudiantes y 7 exámenes diferentes
- Todos los exámenes se realizan en tres días
- Ningún estudiante puede tener más de un exámen por día.
- Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día

In [554]:
import time
import random
from abc import abstractmethod

random.seed(123)

In [555]:
exams = ['Math', 'Physics', 'Chemistry', 'History', 'Biology', 'Geography', 'Literature']
days = ['Monday', 'Tuesday', 'Wednesday']
students_names = ['Francisco', 'Sara', 'Ricardo', 'Fulanito']

In [556]:
class Student:
    def __init__(self, name):
        self.name = name
        self.exams = {'None'}

    def add_exam(self, exam):
        self.exams.add(exam)

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

    def __eq__(self, other):
        return self.name == other.name

    def __hash__(self):
        return hash(self.name)

In [557]:
# Create Students
students = []
for name in students_names:
    students.append(Student(name))

# Assign Exams to Students
for i in range(len(exams)):
    students[i % len(students)].add_exam(exams[i])

# Assign Exams until each student has 3 different exams
for student in students:
    while len(student.exams) < 4:
        additional_exam = random.choice(exams)
        student.add_exam(additional_exam)

# Print students and their exams
print('Students and their exams:')
for student in students:
    print(f'{student}: {[exam for exam in student.exams]}')

Students and their exams:
Francisco: ['None', 'Chemistry', 'Math', 'Biology']
Sara: ['None', 'Geography', 'Physics', 'Math']
Ricardo: ['None', 'Chemistry', 'History', 'Literature']
Fulanito: ['None', 'Chemistry', 'History', 'Math']


In [558]:
variables = []
for student in students:
    for day in days:
        variables.append((student, day))

domains = {}
temp = {}
for var in variables:
    domains[var] = [exam for exam in var[0].exams]
    temp[var] = ""
domains

{(Francisco, 'Monday'): ['None', 'Chemistry', 'Math', 'Biology'],
 (Francisco, 'Tuesday'): ['None', 'Chemistry', 'Math', 'Biology'],
 (Francisco, 'Wednesday'): ['None', 'Chemistry', 'Math', 'Biology'],
 (Sara, 'Monday'): ['None', 'Geography', 'Physics', 'Math'],
 (Sara, 'Tuesday'): ['None', 'Geography', 'Physics', 'Math'],
 (Sara, 'Wednesday'): ['None', 'Geography', 'Physics', 'Math'],
 (Ricardo, 'Monday'): ['None', 'Chemistry', 'History', 'Literature'],
 (Ricardo, 'Tuesday'): ['None', 'Chemistry', 'History', 'Literature'],
 (Ricardo, 'Wednesday'): ['None', 'Chemistry', 'History', 'Literature'],
 (Fulanito, 'Monday'): ['None', 'Chemistry', 'History', 'Math'],
 (Fulanito, 'Tuesday'): ['None', 'Chemistry', 'History', 'Math'],
 (Fulanito, 'Wednesday'): ['None', 'Chemistry', 'History', 'Math']}

In [559]:
temp_list = ['Chemistry', 'Math', 'Biology', 'Geography', 'Physics', 'None', 'History', 'Literature', 'None', 'None', 'None', 'None',] 
for i in range(len(variables)):
    temp[variables[i]] = temp_list[i]

In [560]:
# Constraints
# NOTE THAT, BY THE NATURE OF THE ASSIGNMENT, THE STUDENT CAN'T TAKE MORE THAN ONE EXAM PER DAY
def constraint_all_exams_taken(assignment: dict) -> bool:
    # All values of the exams list must be in the values of the dict
    exams_taken = []
    for var in assignment:
        if assignment[var] != 'None':
            exams_taken.append(assignment[var])
    
    return set(exams) == set(exams_taken)  # If all the exams in the list are part of the assignment, return True

In [561]:
def constraint_one_exam_type_per_day(assignment: dict) -> bool:
    # At max one exam of each type per day
    exams_taken = {}  # day: list[exams]
    for day in days:
        exams_taken[day] = []
    for var in assignment:
        if assignment[var] != 'None':
            if assignment[var] in exams_taken[var[1]]:
                return False
            exams_taken[var[1]].append(assignment[var])
    return True

## Backtracking Search 

In [562]:
def backtracking_search(domains):
    def recursive_backtracking(assignment):
        if len(assignment) == len(domains):
            return assignment

        unassigned = [v for v in domains.keys() if v not in assignment]
        first = unassigned[0]
        for value in domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            if constraint_one_exam_type_per_day(local_assignment):
                result = recursive_backtracking(local_assignment)
                if result is not None:
                    # If the result contains all the exams
                    if constraint_all_exams_taken(result):
                        return result
        return None

    return recursive_backtracking({})

In [563]:
# Perform backtracking search
result = backtracking_search(domains)

In [564]:
for student_day, exam in result.items():
    if exam == 'None':
        continue
    student, day = student_day
    print(f'Student: {student}, Day: {day}, Exam: {exam}')

Student: Francisco, Day: Wednesday, Exam: Biology
Student: Sara, Day: Tuesday, Exam: Geography
Student: Sara, Day: Wednesday, Exam: Physics
Student: Ricardo, Day: Wednesday, Exam: Literature
Student: Fulanito, Day: Monday, Exam: Chemistry
Student: Fulanito, Day: Tuesday, Exam: History
Student: Fulanito, Day: Wednesday, Exam: Math


## Beam Search

## Local Search

# Conclusiones
De cada uno de los algoritmos implementados, tome el tiempo que le toma encontrar una solución, y compare no
solo el tiempo, sino también la solución encontrada de cada uno. 